
# AWS Lab Exercise — IAM Groups, Users, and Custom Policies for EC2 Access Control (Console Based)

### Objective
In this exercise, you will create IAM users and groups to manage access to EC2 instances with varying permissions.  
You will also create a **custom IAM policy** that prevents instance termination, demonstrating granular control over EC2 actions.

---

### Scenario Overview
You will perform the following tasks:

1. Create IAM users `dev01` and `dev02`  
2. Create two IAM groups — `developer` and `developer_readonly`  
3. Assign permissions:
   - `developer` group → `AmazonEC2FullAccess`
   - `developer_readonly` group → `AmazonEC2ReadOnlyAccess`
4. Attach users to respective groups  
5. Verify that `dev01` can create, stop, and terminate instances  
6. Verify that `dev02` can only view instances (read-only)  
7. Create a **custom policy** that *removes* the ability to terminate instances  
8. Attach this custom policy to `developer` group  
9. Re-test access for `dev01` → terminate should now be denied

---

### Architecture Diagram (Conceptual)
```
IAM
 ├── Groups
 │    ├── developer (AmazonEC2FullAccess → then custom restricted policy)
 │    └── developer_readonly (AmazonEC2ReadOnlyAccess)
 │
 ├── Users
 │    ├── dev01 → developer group
 │    └── dev02 → developer_readonly group
 │
 └── EC2 (target resource)
```



## Step 1: Create IAM Users

1. Navigate to **IAM Console** → **Users** → click **Create user**.  
2. **User name:** `dev01`.  
3. Check **Provide user access to the AWS Management Console** → choose **Auto-generate password** → uncheck “Require password reset”.  
4. Click **Next** → skip permission assignment (we’ll assign via group) → click **Create user**.  
5. Repeat the same steps to create another user:  
   - **User name:** `dev02`  
   - Same sign-in options as above.

Both users will now appear under the IAM **Users** list.



## Step 2: Create IAM Groups

1. Go to **IAM Console** → **User groups** → click **Create group**.  
2. **Group name:** `developer`.  
3. On the permissions screen, select the policy **AmazonEC2FullAccess**.  
4. Click **Create group**.  

5. Repeat the process to create another group:  
   - **Group name:** `developer_readonly`  
   - Select policy **AmazonEC2ReadOnlyAccess**  
   - Click **Create group**.

You now have two groups with different access levels to EC2.



## Step 3: Assign Users to Groups

1. Go to **IAM Console** → **Users** → select **dev01**.  
2. Under **Groups**, click **Add user to groups** → select **developer** → click **Add to group**.  
3. Next, select **dev02** → click **Add user to groups** → select **developer_readonly** → click **Add to group**.

Now:
- `dev01` belongs to `developer` (Full EC2 access)  
- `dev02` belongs to `developer_readonly` (Read-only access)



## Step 4: Launch and Test EC2 Permissions for dev01

1. Sign out from the root/admin account.  
2. Sign in as **dev01** (use the IAM login URL from the IAM dashboard).  
3. Navigate to **EC2 → Instances → Launch Instances**.  
4. Launch a small EC2 instance (e.g., Amazon Linux, t2.micro).  
5. Once the instance is running, try the following actions:
   - **Stop instance:** Should succeed ✅  
   - **Start instance:** Should succeed ✅  
   - **Terminate instance:** Should succeed ✅

> `dev01` has full EC2 control because the group is attached to `AmazonEC2FullAccess`.



## Step 5: Launch and Test EC2 Permissions for dev02

1. Sign out from `dev01` and sign in as **dev02**.  
2. Navigate to **EC2 → Instances**.  
3. You should be able to **view instances** and their details but **cannot** start, stop, or terminate them.

> Try clicking **Actions → Instance state** → options will be greyed out or result in an "Access Denied" message.  
> This confirms that **dev02** has read-only EC2 access.



## Step 6: Create a Custom Policy — Deny EC2 Termination

1. Sign back in as **Admin/root user**.  
2. Navigate to **IAM Console** → **Policies** → click **Create policy**.  
3. Choose **JSON** tab and paste the following custom policy:
```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Deny",
            "Action": "ec2:TerminateInstances",
            "Resource": "*"
        }
    ]
}
```
4. Click **Next** → give it a name: `DenyTerminateEC2Instances`.  
5. Add a description (optional): *Prevents instance termination for developer users*.  
6. Click **Create policy**.

This policy explicitly denies instance termination actions.



## Step 7: Attach Custom Policy to Developer Group

1. Go to **IAM Console** → **User groups** → select **developer** group.  
2. Open the **Permissions** tab → click **Add permissions** → **Attach policies**.  
3. Search for your new policy: `DenyTerminateEC2Instances`.  
4. Select it and click **Add permissions**.

Now the `developer` group has both:
- **AmazonEC2FullAccess**
- **DenyTerminateEC2Instances** (explicit deny overrides allow)



## Step 8: Re-Test Access for dev01

1. Sign in again as **dev01**.  
2. Navigate to **EC2 → Instances**.  
3. Try performing the following actions:
   - **Start instance:** Should succeed ✅  
   - **Stop instance:** Should succeed ✅  
   - **Terminate instance:** ❌ **Access Denied**

> AWS IAM evaluation logic gives priority to explicit *Deny* statements.  
> Even though `dev01` inherits full access from `AmazonEC2FullAccess`, the `DenyTerminateEC2Instances` policy overrides it.



---

## Reflection

✅ **What you learned**
- IAM users inherit permissions from their groups.  
- Managed policies (e.g., AmazonEC2FullAccess) provide broad privileges.  
- Custom policies can refine or restrict access as needed.  
- Explicit *Deny* in any policy always takes precedence over *Allow*.  

✅ **Principle of Least Privilege**
Always give users only the permissions they need — no more, no less.

✅ **Key Takeaway**
You successfully implemented a layered IAM design with users, groups, managed and custom policies to enforce granular EC2 access control.

---
